In [1]:
import cv2
import time
import matplotlib.pyplot as plt
from deepface import DeepFace
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import numpy as np


In [17]:
cap = cv2.VideoCapture(0)
frame_count = 0
detected_faces = []
faces = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1
    try:
        if frame_count % 3 == 0:
            faces = DeepFace.extract_faces(frame, detector_backend='mtcnn', enforce_detection=False)
            detected_faces.append(faces)
    except:
        faces = []

    # Draw rectangles for each detected face
    for face_data in faces:
        facial_area = face_data["facial_area"]
        x, y, w, h = facial_area["x"], facial_area["y"], facial_area["w"], facial_area["h"]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, "Osoba", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 
                    0.8, (0, 255, 0), 2, cv2.LINE_AA)
    
    cv2.imshow("Webcam Face Detection", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# detected_faces = []
cap.release()
cv2.destroyAllWindows()

In [18]:
for i, faces in enumerate(detected_faces):
    cropped_face = faces[0]["face"]
    cropped_face_uint8 = (cropped_face * 255).astype(np.uint8)
    cv2.imwrite(f"data/test/target_person/Dominik_{i+714+154}.jpg", cv2.cvtColor(cropped_face_uint8, cv2.COLOR_RGB2BGR))

In [19]:
len(detected_faces)

59

In [4]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from deepface import DeepFace

In [ ]:
input_dir = "data/random_people"      
output_dir = "data/train/non_target_person" 
os.makedirs(output_dir, exist_ok=True)
MAX_DIM = 1024  

for fname in os.listdir(input_dir):
    if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    img_path = os.path.join(input_dir, fname)
    img = cv2.imread(img_path)

    if img is None:
        print(f"Could not read {img_path}")
        continue

    h, w = img.shape[:2]
    scale = MAX_DIM / max(h, w) if max(h, w) > MAX_DIM else 1
    if scale < 1:
        img = cv2.resize(img, (int(w*scale), int(h*scale)))

    try:
        faces = DeepFace.extract_faces(img, detector_backend="mtcnn", enforce_detection=False)
    except Exception as e:
        print(f"Error extracting from {fname}: {e}")
        continue

    if len(faces) == 0:
        print(f"No face found in {fname}")
        continue

    cropped_face = faces[0]["face"]
    cropped_face_uint8 = (cropped_face * 255).astype(np.uint8)

    save_path = os.path.join(output_dir, f"target_{fname}")
    cv2.imwrite(save_path, cv2.cvtColor(cropped_face_uint8, cv2.COLOR_RGB2BGR))
    print(f"Saved {save_path}")

NameError: name 'os' is not defined

In [ ]:
import os
dir1 = "data/train/non_target_person" 
dir2 = "data/val/non_target_person" 
dir3 = "data/test/non_target_person" 
def collect_ids(dir_paths):
    id_dict = {"target": [], "face": []}

    for dir_path in dir_paths:
        for fname in os.listdir(dir_path):
            if fname.endswith(".png"):
                if fname.startswith("target_"):
                    img_id = fname.split("target_")[1].split(".png")[0]
                    id_dict["target"].append(img_id)
                elif fname.startswith("face_"):
                    img_id = fname.split("face_")[1].split(".png")[0]
                    id_dict["face"].append(img_id)
    return id_dict



dirs = [dir1, dir2, dir3]
id_dict = collect_ids(dirs)

print(id_dict)


{'target': ['01274', '01275', '01276', '01277', '01278', '01279', '01280', '01282', '01283', '01284', '01286', '01288', '01289', '01290', '01291', '01292', '01293', '01294', '01295', '01296', '01297', '01298', '01299', '01301', '01303', '01305', '01306', '01307', '01308', '01309', '01310', '01311', '01312', '01313', '01314', '01315', '01317', '01318', '01319', '01320', '01321', '01322', '01323', '01325', '01326', '01327', '01328', '01329', '01330', '01331', '01332', '01335', '01336', '01337', '01339', '01340', '01342', '01344', '01345', '01346', '01347', '01348', '01349', '01350', '01351', '01352', '01353', '01354', '01355', '01357', '01360', '01361', '01362', '01363', '01364', '01365', '01366', '01368', '01369', '01370', '01371', '01372', '01373', '01375', '01376', '01380', '01381', '01382', '01383', '01384', '01385', '01386', '01388', '01389', '01391', '01392', '01393', '01395', '01397', '01398', '01399', '01400', '01401', '01402', '01403', '01404', '01405', '01406', '01407', '01408'

In [ ]:
import os
import cv2
import numpy as np
import random
from deepface import DeepFace

# Already covered IDs
covered_ids = set(id_dict["target"]) | set(id_dict["face"])

input_dir = "data/random_people"
output_dir = "data/test/non_target_person"
os.makedirs(output_dir, exist_ok=True)

MAX_DIM = 1024
NUM_SAMPLES = 1400  # how many random pictures you want


all_files = [f for f in os.listdir(input_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]


id_to_file = {}
for fname in all_files:
    base_name = os.path.splitext(fname)[0]
    img_id = base_name.split("_")[-1]  # assumes format something_<id>.png
    id_to_file[img_id] = fname


available_ids = set(id_to_file.keys()) - covered_ids
print(f"Available IDs after subtracting covered ones: {len(available_ids)}")


sampled_ids = random.sample(list(available_ids), min(NUM_SAMPLES, len(available_ids)))
sampled_files = [id_to_file[i] for i in sampled_ids]
print(len(sampled_files))

Available IDs after subtracting covered ones: 621
621


In [ ]:
import os
import cv2
import random
import numpy as np
from deepface import DeepFace

input_root = r"C:\Users\vxthe\Downloads\vggface2_test\test"
output_dir = r"C:\Users\vxthe\Downloads\vggface2_sampled"
os.makedirs(output_dir, exist_ok=True)

MAX_DIM = 1024
NUM_SAMPLES = 2400


all_files = []
for root, _, files in os.walk(input_root):
    for f in files:
        if f.lower().endswith((".jpg", ".jpeg", ".png")):
            all_files.append(os.path.join(root, f))

print(f"Total images found: {len(all_files)}")


sampled_files = random.sample(all_files, min(NUM_SAMPLES, len(all_files)))
print(f"Randomly chosen {len(sampled_files)} images")


count = 0
for idx, img_path in enumerate(sampled_files, 1):
    img = cv2.imread(img_path)
    if img is None:
        print(f"Could not read {img_path}")
        continue

    
    h, w = img.shape[:2]
    scale = MAX_DIM / max(h, w) if max(h, w) > MAX_DIM else 1
    if scale < 1:
        img = cv2.resize(img, (int(w * scale), int(h * scale)))

    try:
        faces = DeepFace.extract_faces(img, detector_backend="mtcnn", enforce_detection=False)
    except Exception as e:
        print(f"Error extracting from {img_path}: {e}")
        continue

    if len(faces) == 0:
        print(f"No face found in {img_path}")
        continue

    
    cropped_face = faces[0]["face"]
    cropped_face_uint8 = (cropped_face * 255).astype(np.uint8)

    save_path = os.path.join(output_dir, f"face_{count:04d}.png")
    cv2.imwrite(save_path, cv2.cvtColor(cropped_face_uint8, cv2.COLOR_RGB2BGR))
    count += 1
    print(f"[{count}] Saved {save_path}")

    if count >= NUM_SAMPLES:
        print("Reached target of 1400 faces.")
        break

print(f"✅ Done. Extracted {count} faces into {output_dir}")


Total images found: 169396
Randomly chosen 2400 images
[1] Saved C:\Users\vxthe\Downloads\vggface2_sampled\face_0000.png
[2] Saved C:\Users\vxthe\Downloads\vggface2_sampled\face_0001.png
[3] Saved C:\Users\vxthe\Downloads\vggface2_sampled\face_0002.png
[4] Saved C:\Users\vxthe\Downloads\vggface2_sampled\face_0003.png
[5] Saved C:\Users\vxthe\Downloads\vggface2_sampled\face_0004.png
[6] Saved C:\Users\vxthe\Downloads\vggface2_sampled\face_0005.png
[7] Saved C:\Users\vxthe\Downloads\vggface2_sampled\face_0006.png
[8] Saved C:\Users\vxthe\Downloads\vggface2_sampled\face_0007.png
[9] Saved C:\Users\vxthe\Downloads\vggface2_sampled\face_0008.png
[10] Saved C:\Users\vxthe\Downloads\vggface2_sampled\face_0009.png
[11] Saved C:\Users\vxthe\Downloads\vggface2_sampled\face_0010.png
[12] Saved C:\Users\vxthe\Downloads\vggface2_sampled\face_0011.png
[13] Saved C:\Users\vxthe\Downloads\vggface2_sampled\face_0012.png
[14] Saved C:\Users\vxthe\Downloads\vggface2_sampled\face_0013.png
[15] Saved C:\Us

In [3]:
import os

In [2]:
# os.listdir("./data/train/non_target_person")

In [20]:
import os
import random
import shutil

src_dir = "./data/train/non_target_person"
dst_dir = "./data/val/non_target_person"

os.makedirs(dst_dir, exist_ok=True)

files = [f for f in os.listdir(src_dir) if f.startswith("cropped_") and os.path.isfile(os.path.join(src_dir, f))]

random.shuffle(files)
num_to_move = int(0.2 * len(files))
files_to_move = files[:num_to_move]

for f in files_to_move:
    shutil.move(os.path.join(src_dir, f), os.path.join(dst_dir, f))

print(f"Moved {len(files_to_move)} files from {src_dir} to {dst_dir}")


Moved 81 files from ./data/train/non_target_person to ./data/val/non_target_person


In [22]:
import os
import random
import shutil

def move_random_files(src_dir, dst_dir, sample_size):
    os.makedirs(dst_dir, exist_ok=True)
    files = [f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f))]

    if sample_size > len(files):
        raise ValueError(f"Sample size {sample_size} is larger than number of files {len(files)}")

    sampled_files = random.sample(files, sample_size)

    for f in sampled_files:
        shutil.move(os.path.join(src_dir, f), os.path.join(dst_dir, f))

    print(f"Moved {len(sampled_files)} files to {dst_dir}")

src = r"data/train/target_person"
dst = r"data/val/target_person"
n = 140  
move_random_files(src, dst, n)


Moved 140 files to data/val/target_person
